**HIDROLOGIA ESTATÍSTICA  
ATIVIDADE 2 – ESTATÍSTICA DESCRITIVA**  
**Objetivos:** Aplicar os conceitos e ferramentas da estatística descritiva na análise de séries
hidrológicas.  
**Descrição do trabalho:** Selecionar duas bacias hidrográficas. Para cada bacia hidrográfica selecionar uma série
pluviométrica e uma série fluviométrica utilizando a base de dados da Agencia Nacional de
Águas (ANA). Baixar as séries temporais médias em escala diária, mensal e anual para cada
estação. As séries selecionadas devem ter no mínimo 30 anos de extensão sem falhas.
Desenvolver os seguintes itens:
1. Estatística descritiva das séries de vazão em escala diária, mensal e anual. **OK**
2. Estatística descritiva das séries de precipitação em escala mensal e anual. **OK**
3. Apresentar gráficos utilizando a livraria “ggplot2” para apoiar a análise de dados:
- Gráfico de dispersão (Vazão vs precipitação)
- Histograma
- Polígono de frequência
- Box-plots das séries mensais
- Curva de permanência (Série de vazão média anual)
4. Fazer uma discussão crítica que compare a estatística descritiva estimada das variáveis
observadas.
5. Apresentar o trabalho em formato artigo com os seguintes elementos: Resumo,
Introdução, Métodos, Resultados, Conclusões.
O corpo do documento deve ter no máximo 10 páginas. Utilizar anexos de figuras caso for
necessário.

# Funções auxiliares

In [3]:
import pandas as pd
import activ_2 as aux
salvar = False

# Estatísticas descritivas

In [7]:
df_tiririca = pd.read_excel(
    'org_geral.xlsx', sheet_name='CAM', usecols='A:C',
    header=0, index_col=0, names=['data', 'fluv', 'pluv'],
)

df_fazendinha = pd.read_excel(
    'org_geral.xlsx', sheet_name='SJP', usecols='A:C',
    header=0, index_col=0, names=['data', 'fluv', 'pluv'],
)

## Fazendinha

### Pluviométrica

In [8]:
df = df_tiririca.pluv
i = df.index.min().date()
f = df.index.max().date()


In [10]:
tiririca_pluv = aux.activ_2_stats(df, i , f)

ValueError: array length 21185 does not match index length 17533

In [14]:
df

data
1964-06-01    NaN
1964-06-02    NaN
1964-06-03    NaN
1964-06-04    NaN
1964-06-05    2.0
             ... 
NaT           NaN
NaT           NaN
NaT           NaN
NaT           NaN
NaT           NaN
Name: pluv, Length: 21946, dtype: float64

### Fluviométrica

In [9]:
df = df_tiririca.fluv
i = df.index.min().date()
f = df.index.max().date()
tiririca_fluv = aux.activ_2_stats(df, i , f)

## Tiririca

### Pluviométrica

In [12]:
df = df_fazendinha.pluv
i = df.index.min().date()
f = df.index.max().date()
fazendinha_pluv = aux.activ_2_stats(df, i , f)

ValueError: array length 21064 does not match index length 21946

### Fluviométrica

In [11]:
df = df_fazendinha.fluv
i = df.index.min().date()
f = df.index.max().date()
fazendinha_fluv = aux.activ_2_stats(df, i , f)

ValueError: array length 21064 does not match index length 21946

# Geração de gráficos

### Gráfico de dispersão

In [ ]:
fig, ax = plt.subplots()

ax.scatter(df_ano[0]["data"], df_ano[1]["data"] * 365)
ax.set_xlabel("Vazão (m$^3$/s)")
ax.set_ylabel("Precipitação (mm/ano)")

if salvar:
    fig.savefig("figures/scatter_sjp.png", format="png", dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.scatter(df_ano[2]["data"].iloc[1:-1], df_ano[3]["data"].iloc[1:-1] * 365)
ax.set_xlabel("Vazão (m$^3$/s)")
ax.set_ylabel("Precipitação (mm/ano)")

if salvar:
    fig.savefig("figures/scatter_sjp.png", format="png", dpi=300)
plt.show()

### Histograma

In [ ]:
serie_anual = df_ano[0]
N = serie_anual.shape[0]
NC = round(1 + 3.3 * np.log10(N))  # Sturges

In [ ]:
counts, bins_pol = np.histogram(serie_anual["data"], bins=NC)

In [ ]:
# Frequency polygon
counts, bins_pol = np.histogram(serie_anual["data"], bins=NC)
bins_pol = np.concatenate([np.array([bins_pol[0]]), (bins_pol[:-1] + bins_pol[1:]) / 2, np.array([bins_pol[-1]])])
counts = np.concatenate([np.zeros(1), counts, np.zeros(1)])

q1, q3 = np.percentile(serie_anual["data"], [25, 75])
whisker_low = q1 - (q3 - q1) * 1.5
whisker_high = q3 + (q3 - q1) * 1.5
outliers = serie_anual["data"][(serie_anual["data"] > whisker_high) | (serie_anual["data"] < whisker_low)]

fig, ax = plt.subplots(2, 1, constrained_layout=True, gridspec_kw={"height_ratios":[2, 1]})
ax_right = ax[0].twinx()

n, bins, _ = ax[0].hist(
        x=serie_anual["data"], bins='sturges', color='#36759f', edgecolor='#3d3d3d',
)
ax[0].set_ylabel("Freq. Absoluta")
ax[0].set_xticks(bins)
ax[0].set_xticklabels(bins.round())
ax_right.hist(
        x=serie_anual["data"], bins=len(n), color='#36759f', edgecolor='#3d3d3d', 
        weights=np.ones_like(serie_anual["data"].to_numpy())/len(serie_anual["data"]),
)
ax_right.set_ylabel("Freq. Relativa")

ax_right.plot(bins_pol, counts/len(serie_anual["data"]), '--', color='#3d3d3d')

sns.violinplot(x=serie_anual["data"], cut=0, ax=ax[1])
ax[1].set_xticks(bins)
ax[1].set_xticklabels(bins.round())
ax[1].set_yticks([])
ax[1].set_xlabel('Vazão (m$^3$/s)')
ax[1].set_ylabel("Violin Plot")
sns.scatterplot(x=outliers, y=0, marker='D', edgecolor='#3d3d3d', color='#3d3d3d', ax=ax[1])

if salvar:
    fig.savefig('report/plots.png', format='png', dpi=300)
plt.plot()

### Box-plot

In [ ]:
df = {}
for m in range(1, 13):
    x = df_mes[0][df_mes[0]["mes"] == m]["data"]
    df[m] = x.values
df = pd.DataFrame(df)

In [ ]:
fig, ax = plt.subplots(constrained_layout=True, figsize=(8, 4))

sns.violinplot(data=df, cut=0, ax=ax)

for m in range(1, 13):
    q1, q3 = np.percentile(df[m], [25, 75])
    whisker_low = q1 - (q3 - q1) * 1.5
    whisker_high = q3 + (q3 - q1) * 1.5
    outliers = df[m][(df[m] > whisker_high) | (df[m] < whisker_low)]
    sns.scatterplot(x)

ax.plot(range(12), df.mean(), "--", color='red', alpha=0.75, label="Média")
    
ax.set_xlabel('Mês')
ax.set_ylabel('Vazão (m$^3$/s)')
ax.legend()
=m-1, y=outliers, marker='D', edgecolor='#3d3d3d', color='#3d3d3d', ax=ax
if salvar:
    fig.savefig('report/plots.png', format='png', dpi=300)
plt.plot()

### Curva de permanência

In [ ]:
x = df_ano[0]["data"]

In [ ]:
# Flow duration curve
vazao_decr = x.sort_values(ascending=False)
N = vazao_decr.shape[0]
n = np.arange(N) + 1
n_N = n / N

fig, ax = plt.subplots()
ax.plot(n_N * 100, vazao_decr, "--")

ax.fill_between(n_N * 100, vazao_decr, alpha=0.2)

ax.set_xlabel('Permanência (%)')
ax.set_xlim([n_N.min() * 100, 100])
ax.set_ylabel('Vazão (m$^3$/s)')
ax.set_ylim([0, vazao_decr.max()])

if salvar:
    fig.savefig()
plt.show()